In [32]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import re


# 예외 문제 제목 리스트
exceptions = ["A/B", "X", "Y", "DPS", "XOR-ABC", "Great Pow!", "What's your ETA?", "Base Conversion", "200% Mixed Juice!", "The candy war", ":danceplant:", "solved.ac", "ABCDE" "CCW", "Count Circle Groups", "MooTube (Silver)", "Puyo Puyo", "Your life", "Yes or yes", "Small World Network", "DSLR", "Count Circle Groups", "Three Dots",  "A/B - 2", "△", "A+B - 6", "HI-ARC=?", "Hello World!", "2048 (Hard)", "NN","N-Queen", "Fan Death", "Baaaaaaaaaduk2 (Easy)", "IQ Test", "Maaaaaaaaaze", "Gaaaaaaaaaarden", "⚾", "Maximum Subarray", "369", "2048 (Easy)", "Send me the money", "SMUPC NAME", "driip", "iSharp", "ZOAC 5", "ZOAC 3", "Since 1973", "Serious Problem", "IPv6", "ZOAC", "6174", "Quicksum", "LCD Test", "FizzBuzz", "Cubeditor", "Java vs C++", "Welcome to SMUPC!", "MBTI", "PPAP", "LCS", "LCS 2", "LCS 3", "30", "FBI", "Contact", "Strfry", "IOIOI", "ROT13", "Hashing", "A+B−C", "LCS 4", "LCS 5", "Escaping","Hanyang Popularity Exceeding Competition","ATM","A → B","Number Game","Project Teams", "Yonsei TOTO", "NMK", "Byte Coin ", "ZOAC 2", "Pen Pineapple Apple Pen", "Double It ", "DNA", "A×B", "A-B", "A+B", "Hello World", "A+B - 4", "A+B - 5", "A+B - 6", "A+B - 2", "A+B - 8", "A+B - 7", "A+B - 5", "A+B - 3", "ZOAC 4", "GCD(n, k) = 1"]


# 웹 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")  # 브라우저를 최대화
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

wait = WebDriverWait(driver, 10)

# 수집할 데이터 리스트
data = []

# 문제 번호 추출 패턴
problem_num_pattern = re.compile(r'problem/(\d+)')
korean_pattern = re.compile(r'[가-힣]')  # 한글 확인용 정규식

# 1~50 페이지 반복(알고리즘 타입에 따라)
for page in range(1, 5):
    print(f"현재 페이지: {page}")  # 디버깅용 페이지 번호 출력
    url = f"https://solved.ac/problems/tags/combinatorics?sort=solved&direction=desc&page={page}" # 이건 조함론 링크이고, 본인이 수집하고 싶은 알고리즘 타입으로 url을 바꿔주세요.
    driver.get(url)
    time.sleep(1)  # 페이지 로딩 대기
    
    # 문제 리스트 찾기
    problems = driver.find_elements(By.CSS_SELECTOR, '.css-1ojb0xa')  # 문제 전체 셀렉터 수정
    print(f"문제 수집 시도: {len(problems)}개의 문제 발견")  # 문제 리스트 수 출력

    for problem in problems:
        try:
            # 잠금 아이콘이 있는지 확인
            lock_icon = problem.find_elements(By.CSS_SELECTOR, 'img[alt="lock"]')
            if lock_icon:
                print("잠금된 문제 건너뜀")  # 디버깅용 잠금된 문제 알림
                continue  # 잠금된 문제는 건너뜀
        except:
            pass  # 잠금 아이콘이 없으면 계속 진행

        # 문제 번호 추출
        try:
            problem_link = problem.find_element(By.CSS_SELECTOR, 'a.css-q9j30p')
            problem_url = problem_link.get_attribute('href')
            problem_num_match = problem_num_pattern.search(problem_url)
            if problem_num_match:
                problem_num = problem_num_match.group(1)
                print(f"문제 번호: {problem_num}")  # 디버깅용 문제 번호 출력
        except Exception as e:
            print(f"문제 번호를 찾을 수 없습니다: {e}")
            continue

        # 문제 티어 추출
        try:
            tier_img = problem.find_element(By.CSS_SELECTOR, 'img')
            problem_tier = tier_img.get_attribute('alt')
            print(f"문제 티어: {problem_tier}")  # 디버깅용 티어 출력
        except Exception as e:
            print(f"문제 티어를 찾을 수 없습니다: {e}")
            continue

        # 문제 제목 추출
        try:
            problem_title_elem = problem.find_element(By.CSS_SELECTOR, 'span.__Latex__')
            problem_title = problem_title_elem.text.strip()
            print(f"문제 제목: {problem_title}")  # 디버깅용 문제 제목 출력
        except Exception as e:
            print(f"문제 제목을 찾을 수 없습니다: {e}")
            continue

        # 제목이 한글이 아니면서 예외 리스트에 없는 경우 건너뜀
        if not korean_pattern.search(problem_title) and problem_title not in exceptions:
            print(f"한글이 아니면서 예외 목록에 없는 문제 제목 건너뜀: {problem_title}")  # 디버깅용 한글 문제 필터링
            continue

        # 푼 사람 수 추출 (콤마 제거)
        try:
            solved_count_elem = problem.find_element(By.CSS_SELECTOR, 'div.css-1ujcjo0')
            solved_count = solved_count_elem.text.strip().replace(",", "")  # 콤마 제거
            print(f"푼 사람 수: {solved_count}")  # 디버깅용 푼 사람 수 출력
        except Exception as e:
            print(f"푼 사람 수를 찾을 수 없습니다: {e}")
            continue

        # 문제 링크 생성
        problem_link = f"https://www.acmicpc.net/problem/{problem_num}"
        print(f"문제 링크: {problem_link}")  # 디버깅용 문제 링크 출력

        # 수집한 데이터 저장
        data.append([problem_num, problem_tier, problem_title, solved_count, problem_link])

# 데이터프레임 생성
df = pd.DataFrame(data, columns=['문제 번호', '문제 티어', '문제 이름', '푼 사람 수', '문제 링크'])

# CSV 파일로 저장(다만 이게 dfs 면 파일명 바꿔줘야함.)
df.to_csv('조합론_대표문제.csv', index=False)
print("CSV 파일 저장 완료")  # 디버깅용 CSV 저장 완료 알림

# 브라우저 종료
driver.quit()

print("브라우저 종료 완료")  

현재 페이지: 1
문제 수집 시도: 51개의 문제 발견
문제 번호를 찾을 수 없습니다: Message: no such element: Unable to locate element: {"method":"css selector","selector":"a.css-q9j30p"}
  (Session info: chrome=129.0.6668.59); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x006D83E3+25571]
	(No symbol) [0x0066A684]
	(No symbol) [0x00562113]
	(No symbol) [0x005A6FB2]
	(No symbol) [0x005A71FB]
	(No symbol) [0x0059D161]
	(No symbol) [0x005CAC54]
	(No symbol) [0x0059D084]
	(No symbol) [0x005CAEA4]
	(No symbol) [0x005E5349]
	(No symbol) [0x005CA9A6]
	(No symbol) [0x0059BAB6]
	(No symbol) [0x0059C50D]
	GetHandleVerifier [0x009AC4A3+2991267]
	GetHandleVerifier [0x009FD2C9+3322569]
	GetHandleVerifier [0x007684D2+615634]
	GetHandleVerifier [0x0076FBFC+646140]
	(No symbol) [0x0067327D]
	(No symbol) [0x00670188]
	(No symbol) [0x00670325]
	(No symbol) [0x00662826]
	BaseThreadInitThunk [0x7570FCC9+25]
	

In [33]:
# 문제 정답률 등은 solved.ac 에서 바로 가져오는게 불가능하고, 문제 링크를 따로 타고 들어가서 가져와야 합니다.

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

wait = WebDriverWait(driver, 10)

df = pd.read_csv('조합론_대표문제.csv')

# 새 데이터를 추가할 리스트
additional_data = []

# 문제 링크를 순차적으로 방문하여 추가 데이터를 수집
for index, row in df.iterrows():
    driver.get(row['문제 링크'])
    # time.sleep(0.1)  # 페이지 로딩 대기

    # 전체 제출 케이스, 정답 케이스, 정답 비율 수집
    try:
        tbody = driver.find_element(By.CSS_SELECTOR, 'tbody')
        tds = tbody.find_elements(By.TAG_NAME, 'td')

        total_submissions = tds[2].text.strip().replace(",", "")  # 전체 제출 케이스
        correct_submissions = tds[3].text.strip().replace(",", "")  # 정답 케이스
        correct_rate = tds[5].text.strip()  # 정답 비율 (소수점 포함)
        print(f"total_submissions: {total_submissions}, correct_submissions: {correct_submissions}, correct_rate: {correct_rate}")
        # 수집한 데이터를 리스트에 저장
        additional_data.append([total_submissions, correct_submissions, correct_rate])

    except Exception as e:
        print(f"Error collecting data for problem {row['문제 번호']}: {e}")
        additional_data.append([None, None, None])  # 오류가 발생한 경우 빈 값으로 처리

# 추가 데이터를 기존 데이터프레임에 열로 추가
df['전체 제출 케이스'] = [data[0] for data in additional_data]
df['정답 케이스'] = [data[1] for data in additional_data]
df['정답 비율'] = [data[2] for data in additional_data]

df = df[['문제 번호', '문제 티어', '문제 이름', '전체 제출 케이스', '정답 케이스', '푼 사람 수', '정답 비율', '문제 링크']]

# 골드 3 유저가 풀수 없는 티어의 문제 제거
tiers_to_remove = ["Unrated", "Platinum I", "Platinum II", "Diamond I", "Diamond II", "Diamond III", "Diamond IV", "Diamond V", 
                   "Ruby I", "Ruby II", "Ruby III", "Ruby IV", "Ruby V", "Master"]

filtered_df = df[~df['문제 티어'].isin(tiers_to_remove)]
filtered_df.reset_index(drop=True, inplace=True)

filtered_df.to_csv('조합론_대표문제.csv', index=False)
print("CSV 파일 저장 완료")
# 브라우저 종료
driver.quit()

print("추가 데이터 수집 완료 및 CSV 저장 완료!")


total_submissions: 110268, correct_submissions: 51570, correct_rate: 47.928%
total_submissions: 72533, correct_submissions: 47111, correct_rate: 64.769%
total_submissions: 81607, correct_submissions: 39062, correct_rate: 44.810%
total_submissions: 63391, correct_submissions: 36094, correct_rate: 55.751%
total_submissions: 44701, correct_submissions: 24967, correct_rate: 55.613%
total_submissions: 63389, correct_submissions: 24131, correct_rate: 38.329%
total_submissions: 15649, correct_submissions: 13449, correct_rate: 86.504%
total_submissions: 34942, correct_submissions: 13361, correct_rate: 42.422%
total_submissions: 32474, correct_submissions: 12976, correct_rate: 40.461%
total_submissions: 31495, correct_submissions: 11589, correct_rate: 39.716%
total_submissions: 30273, correct_submissions: 13320, correct_rate: 40.479%
total_submissions: 16167, correct_submissions: 8976, correct_rate: 58.330%
total_submissions: 18931, correct_submissions: 7047, correct_rate: 37.235%
total_submiss